In [1]:
from helperutil import file_date_util as util
import sys
import os
sys.path.insert(0, './dags')

from restaurant_pipeline import default_args
from restaurant_pipeline import get_process_file_list

/home/developer/python-dev/airflow_project1/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


[2018-09-18 22:52:50,691] {__init__.py:57} INFO - Using executor SequentialExecutor
[2018-09-18 22:52:51,132] {driver.py:124} INFO - Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
[2018-09-18 22:52:51,186] {driver.py:124} INFO - Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
[2018-09-18 22:52:52,622] {restaurant_pipeline.py:154} INFO - today's downloading files: restaurant_data_2018_09_14.json
[2018-09-18 22:52:52,623] {restaurant_pipeline.py:154} INFO - today's downloading files: restaurant_data_2018_09_12.json
[2018-09-18 22:52:52,624] {restaurant_pipeline.py:154} INFO - today's downloading files: restaurant_data_2018_09_15.json
[2018-09-18 22:52:52,626] {restaurant_pipeline.py:154} INFO - today's downloading files: restaurant_data_2018_09_13.json


In [3]:
import pandas as pd
from datetime import date as d

In [4]:
from sqlalchemy import create_engine

In [5]:
# # setup default arguments
# default_args = {
#     'download_url': 'https://data.cityofnewyork.us/resource/9w7m-hzhe.json?inspection_date=',
#     'download_date_days_ago': -3,
#     'download_date_months_ago': -3,
#     'db_hostname': '192.168.1.221',
#     'db_username': 'restuser',
#     'db_password': 'restuser',
#     'db_database': 'restaurantdb',
# }


# setup default arguments
default_args = {
    'download_url': 'https://data.cityofnewyork.us/resource/9w7m-hzhe.json?inspection_date=',
    'download_date_days_ago': -3,
    'download_date_months_ago': -4,
    'db_hostname': 'localhost',
    'db_username': 'root',
    'db_password': 'workadmin',
    'db_database': 'restaurant_inbox',
}



In [6]:
engine = create_engine("mysql+pymysql://" + default_args['db_username'] + ":" + default_args['db_password'] + "@" + default_args['db_hostname'] + "/" + default_args['db_database'] + "", echo=False)
conn = engine.connect()

In [42]:
# files_to_process = get_process_file_list()
# dataframes = (pd.read_json(f, lines=True, encoding='UTF8') for f in files_to_process)
# big_dataframe = pd.concat(dataframes, ignore_index=True)
# # big_dataframe = big_dataframe.reset_index()

sql_string = """
select distinct
v_camis as camis, v_grade as grade, v_grade_date as grade_date 
from restaurant_inbox
"""

df_grade = pd.read_sql(sql_string, conn)
big_dataframe = df_grade

In [51]:
df_grade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110415 entries, 0 to 110414
Data columns (total 3 columns):
camis         110414 non-null float64
grade         110319 non-null object
grade_date    110316 non-null object
dtypes: float64(1), object(2)
memory usage: 2.5+ MB


In [53]:
# downloading_list
# path_name

# date_end
# date_start

#big_dataframe.head(2)
big_dataframe[big_dataframe['grade_date'] != '0000-00-00 00:00:00'].head(10)

,camis,grade,grade_date
3,50041630.0,A,2018-01-31 00:00:00
4,41130552.0,A,2018-01-31 00:00:00
11,50008339.0,A,2018-01-31 00:00:00
12,50017896.0,A,2018-01-31 00:00:00
18,41717443.0,A,2018-01-31 00:00:00
21,41249587.0,A,2018-01-31 00:00:00
25,50041181.0,A,2018-01-31 00:00:00
28,50003103.0,A,2018-01-31 00:00:00
32,50004139.0,Z,2018-01-31 00:00:00
33,50053806.0,Z,2018-01-31 00:00:00


In [55]:
big_dataframe = big_dataframe[big_dataframe['grade_date'] != '0000-00-00 00:00:00']
big_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87249 entries, 3 to 110414
Data columns (total 3 columns):
camis         87248 non-null float64
grade         87153 non-null object
grade_date    87150 non-null object
dtypes: float64(1), object(2)
memory usage: 2.7+ MB


In [56]:
big_dataframe = big_dataframe[['camis', 'grade', 'grade_date']].drop_duplicates(subset=['camis', 'grade', 'grade_date'])
big_dataframe = big_dataframe[big_dataframe['grade_date'].isna() == False]
big_dataframe.columns = ['business_number', 'grade', 'grade_date']


In [57]:
#df_grade = big_dataframe
big_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87150 entries, 3 to 110414
Data columns (total 3 columns):
business_number    87150 non-null float64
grade              87150 non-null object
grade_date         87150 non-null object
dtypes: float64(1), object(2)
memory usage: 2.7+ MB


In [60]:
df_grade = big_dataframe
df_grade.head()

,business_number,grade,grade_date
3,50041630.0,A,2018-01-31
4,41130552.0,A,2018-01-31
11,50008339.0,A,2018-01-31
12,50017896.0,A,2018-01-31
18,41717443.0,A,2018-01-31


In [59]:
df_grade['grade_date'] = df_grade['grade_date'].astype('datetime64')
# s[s.isna() == True].value_counts()
# s.value_counts

In [37]:
# df_grade.head(10)
df_grade = df_grade[df_grade['grade_date'].isna() == False]

In [38]:
#df_grade.head(10)
df_grade = df_grade[df_grade['grade_date'].isna() == False]

In [62]:
df_grade[df_grade.isna() == False].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87150 entries, 3 to 110414
Data columns (total 3 columns):
business_number    87150 non-null float64
grade              87150 non-null object
grade_date         87150 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 2.7+ MB


In [63]:
truncate_sql = """
truncate table trans_dim_grade
"""
update_res = pd.io.sql.execute(truncate_sql, conn)

In [64]:
df_grade.to_sql('trans_dim_grade', con=engine, if_exists='append', index=False)

In [65]:
delete_sql = """
delete a
from trans_dim_grade a left join dim_grade b on a.business_number = b.business_number and a.grade_date = b.grade_date
where b.business_number is not null
"""
delete_res = pd.io.sql.execute(delete_sql, conn)

In [66]:
insert_sql = """
insert into dim_grade (business_number, grade, grade_date, create_date, modify_date)
select business_number, grade, grade_date, now(), now() from trans_dim_grade
"""
update_res = pd.io.sql.execute(insert_sql, conn)